# Práctica 8: Mapas

In [5]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium

In [6]:
df = pd.read_excel('Canada.xls', sheet_name='Canada by Citizenship (2)')

In [7]:
def basic_cleaning(df):
    cleaned_df = df.copy()
    cleaned_df = cleaned_df.drop(columns=['Type', 'Coverage', 'AREA', 'AreaName', 'REG', 'RegName', 'DEV', 'DevName'])
    cleaned_df = cleaned_df.rename(columns={'OdName':'Country'})
    cleaned_df['Total'] = cleaned_df.iloc[:, 1:].agg('sum', axis=1)
    return cleaned_df

## Gráficos de área 

In [8]:
area_df = basic_cleaning(df)
area_df = area_df.sort_values(by='Total', ascending=False)
area_df = area_df.drop(area_df.index[3])
area_df = area_df.iloc[:5, :]
area_df = area_df.drop(columns='Total')
area_df = area_df.set_index('Country')
area_df = area_df.T
area_df = area_df.reset_index()
area_df = area_df.rename(columns={
    'United Kingdom of Great Britain and Northern Ireland':'Reino Unido e Irlanda del Norte', 'index':'Year',
    'Philippines':'Filipinas','Pakistan':'Pakistán'
    })
area_df.head()

Country,Year,India,China,Reino Unido e Irlanda del Norte,Filipinas,Pakistán
0,1980,8880,5123,22045,6051,978
1,1981,8670,6682,24796,5921,972
2,1982,8147,3308,20620,5249,1201
3,1983,7338,1863,10015,4562,900
4,1984,5704,1527,10170,3801,668


In [9]:
def plot_area_chart(df, colors):
    fig = go.Figure()
    countries = list(df.columns)[1:]
    for idx, country in enumerate(countries):
        fig.add_trace(go.Scatter(
            x=df.Year, y=df[country], mode='lines',
            line=dict(width=1.1, color=colors[country]),
            stackgroup='one', groupnorm='percent',
            text=country, name=country
        ))
    fig.update_layout(
        title=dict(
            text='Inmigración desde los 5 países con mayor número de migrantes hacía Canadá',
            font=dict(size=16),
            x=0.75, y=0.87
        ),
        margin=dict(
            pad=10
        ),
        width=800, height=510, template='plotly_white',
        showlegend=False
    )
    ys = [10, 42, 60, 80, 93]
    xs = [2006, 2009, 1987, 2010, 2002]
    for idx, country in enumerate(countries):
        fig.add_annotation(
            x = xs[idx],
            y = ys[idx],
            text = f'<b>{country}<b>',
            showarrow=False,
            font=dict(
                color="#eff1f4"
            ),
            borderpad=4,
            bgcolor=colors[country],
            opacity=0.8
        )
    return fig

In [10]:
colors = {
    'India':'#f94144', 'China':'#f8961e', 
    'Reino Unido e Irlanda del Norte':'#90be6d', 
    'Filipinas':'#43aa8b', 'Pakistán':'#577590'
}
plot_area_chart(area_df, colors).show()

In [11]:
def plot_individual_areas(df, colors):
    countries = list(df.columns)[1:]
    fig = make_subplots(rows=3, cols=2, subplot_titles=list(colors.keys()))
    row, col = 1, 1
    for idx, country in enumerate(countries):
        fig.add_trace(
            go.Scatter(
                x=df.Year, y=df[country], mode='lines',
                line=dict(width=1.3, color=colors[country]),
                stackgroup='one', name=country
            ),
            row=row, col=col
        )
        fig.update_layout(
            title=dict(
                text='Tendencias de inmigración por país hacía Canadá',
                font=dict(size=19)
            ),
            margin=dict(
                pad=10
            ),
            width=800, height=800, showlegend=False, template='plotly_white'
        )
        if col < 2: col += 1
        else: 
            row += 1
            col = 1

    return fig 

In [12]:
plot_individual_areas(area_df, colors).show()

## Gráficos de barras verticales

In [13]:
def migration_per(country, df):
    migration_df = basic_cleaning(df)
    migration_df = migration_df.drop(columns='Total')
    migration_df = migration_df[migration_df.Country == country]
    migration_df = migration_df.set_index('Country')
    migration_df = migration_df.T
    migration_df = migration_df.reset_index()
    migration_df = migration_df.rename(columns={'index':'Year', country:'Migrants'})
    return migration_df

In [14]:
iceland_df = migration_per('Iceland', df)
pre_crisis_df = iceland_df[iceland_df.Year < 2008]
crisis_df = iceland_df[iceland_df.Year >= 2008]
increase_rate = round(int(crisis_df[crisis_df.Year == 2013].Migrants)*100/int(crisis_df[crisis_df.Year == 2008].Migrants),2)
print(increase_rate)

553.85


In [15]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=pre_crisis_df.Year, y=pre_crisis_df.Migrants,
    marker_color='lightslategrey', name=''
))
fig.add_trace(go.Bar(
    x=crisis_df.Year, y=crisis_df.Migrants,
    marker_color='crimson', name=''
))
fig.add_trace(go.Scatter(
    x=[2008,2012.7], y=[15,77],
    mode='lines+markers+text', 
    marker_color='crimson'
))
fig.add_annotation(
    x=2009.8,
    y=76.7,
    text="<b>Crisis económica de <br> 2008-2011</b>",
    showarrow=False,
    font=dict(
        color='crimson'
    )
)
fig.add_annotation(
    x=2008.2,
    y=42,
    text="Número de migrantes <br> incrementó en un <br> 554%",
    showarrow=True,
    arrowhead=1,
    arrowsize=1,
    ax=-20,
    ay=-30,
    arrowcolor='black',
    font=dict(
        color='black'
    )
)
fig.update_layout(
    title=dict(
        text='Migrantes provenientes de Islandia hacía Canadá',
        font=dict(size=16, color='black')
    ),
    margin=dict(
        pad=10
    ),
    width=1000, height=600, template='plotly_white', showlegend=False
)
fig.show()

In [16]:
argentina_df = migration_per('Argentina', df)
pre_df = argentina_df[argentina_df.Year < 1998]
mid_df = argentina_df[(argentina_df.Year >= 1998) & (argentina_df.Year < 2004)]
post_df = argentina_df[argentina_df.Year >= 2004]
increase_rate = round(int(argentina_df[argentina_df.Year == 2004].Migrants)*100/int(argentina_df[argentina_df.Year == 1998].Migrants),2)
print(increase_rate)

455.87


In [17]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=pre_df.Year, y=pre_df.Migrants,
    marker_color='lightslategrey', name=''
))
fig.add_trace(go.Bar(
    x=mid_df.Year, y=mid_df.Migrants,
    marker_color='crimson', name=''
))
fig.add_trace(go.Bar(
    x=post_df.Year, y=post_df.Migrants,
    marker_color='lightslategrey', name=''
))
fig.add_trace(go.Scatter(
    x=[1998, 2002.73], y=[349+50,1745+130],
    mode='lines+markers+text', 
    marker_color='crimson'
))
fig.add_annotation(
    x=2000,
    y=1745+80,
    text="<b>Crisis económica de <br> 1998-2003</b>",
    showarrow=False,
    font=dict(
        color='crimson'
    )
)
fig.add_annotation(
    x=1999,
    y=1100,
    text="Número de migrantes <br> incrementó en un <br> 456%",
    showarrow=True,
    arrowhead=1,
    arrowsize=1,
    ax=-20,
    ay=-30,
    arrowcolor='black',
    font=dict(
        color='black'
    )
)
fig.update_layout(
    title=dict(
        text='Migrantes provenientes de Argentina hacía Canadá',
        font=dict(size=16, color='black')
    ),
    margin=dict(
        pad=10
    ),
    width=1000, height=600, template='plotly_white', showlegend=False
)
fig.show()

## Mapas coropléticos

In [18]:
world_df = basic_cleaning(df)
world_df = world_df.loc[:, ['Country','Total']]
world_df.head()

,Country,Total
0,Afghanistan,58639
1,Albania,15699
2,Algeria,69439
3,American Samoa,6
4,Andorra,15


In [28]:
len(world_df.Country.unique())

196

In [20]:
immigration_data = 'world_countries.json'
m = folium.Map(location=[20, 0], zoom_start=1.5, width=1000, height=600)
folium.Choropleth(
    immigration_data,
    name="choropleth",
    data = world_df,
    columns=["Country", "Total"],
    key_on="feature.properties.name",
    fill_color="RdPu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Inmigración hacía Canadá entre 1980 y 2013",
    nan_fill_opacity=0,
    bins=3,
).add_to(m)
folium.LayerControl().add_to(m)
m

In [21]:
eighties_df = basic_cleaning(df)
eighties_df = eighties_df.iloc[:, :11]
eighties_df['Total'] = eighties_df.iloc[:, 1:].agg('sum', axis=1)
eighties_df = eighties_df.loc[:, ['Country', 'Total']]
immigration_data = 'world_countries.json'
m = folium.Map(location=[20, 0], zoom_start=1.5, width=1000, height=600)
folium.Choropleth(
    immigration_data,
    name="choropleth",
    data = eighties_df,
    columns=["Country", "Total"],
    key_on="feature.properties.name",
    fill_color="Reds",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Inmigración hacía Canadá entre 1980-1989",
    nan_fill_opacity=0,
    bins=3,
).add_to(m)
folium.LayerControl().add_to(m)
m

In [22]:
nineties_df = basic_cleaning(df)
nineties_df = nineties_df.iloc[:, [0]+list(range(11,21))]
nineties_df['Total'] = nineties_df.iloc[:, 1:].agg('sum', axis=1)
nineties_df = nineties_df.loc[:, ['Country', 'Total']]
immigration_data = 'world_countries.json'
m = folium.Map(location=[20, 0], zoom_start=1.5, width=1000, height=600)
folium.Choropleth(
    immigration_data,
    name="choropleth",
    data = nineties_df,
    columns=["Country", "Total"],
    key_on="feature.properties.name",
    fill_color="Reds",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Inmigración hacía Canadá entre 1990-1999",
    nan_fill_opacity=0,
    bins=3,
).add_to(m)
folium.LayerControl().add_to(m)
m

In [23]:
thousands_df = basic_cleaning(df)
thousands_df = thousands_df.iloc[:, [0]+list(range(21,31))]
thousands_df['Total'] = thousands_df.iloc[:, 1:].agg('sum', axis=1)
thousands_df = thousands_df.loc[:, ['Country', 'Total']]
immigration_data = 'world_countries.json'
m = folium.Map(location=[20, 0], zoom_start=1.5, width=1000, height=600)
folium.Choropleth(
    immigration_data,
    name="choropleth",
    data = thousands_df,
    columns=["Country", "Total"],
    key_on="feature.properties.name",
    fill_color="Reds",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Inmigración hacía Canadá entre 2000-2009",
    nan_fill_opacity=0,
    bins=3,
).add_to(m)
folium.LayerControl().add_to(m)
m

## Otros dos mapas

In [24]:
internet_df = pd.read_csv('share-of-individuals-using-the-internet.csv', index_col='Entity')
internet_df = internet_df[internet_df.Year == 2016]
internet_df = internet_df.rename(columns={'Individuals using the Internet (% of population)':'Population'})
internet_df = internet_df.drop(columns=['Code', 'Year'])
internet_df = internet_df.rename(index={'United States':'United States of America', 'Russia':'Russian Federation'})
internet_df = internet_df.reset_index()
internet_df = internet_df.rename(columns={'Entity':'Country'})
internet_df.head()

,Country,Population
0,Afghanistan,10.595726
1,Albania,66.363445
2,Algeria,42.945527
3,Andorra,97.930637
4,Angola,13.000000


In [25]:
world_data = 'world_countries.json'
m = folium.Map(location=[40, 0], zoom_start=2, width=1000, height=600)
folium.Choropleth(
    world_data,
    name="choropleth",
    data = internet_df,
    columns=["Country", "Population"],
    key_on="feature.properties.name",
    fill_color="YlGnBu",
    fill_opacity=0.75,
    line_opacity=0.2,
    legend_name="Porcentaje de población que utilizó el internet en 2016",
    nan_fill_opacity=0.25,
    bins=5,
    highlight=True
).add_to(m)
folium.LayerControl().add_to(m)
m

https://ourworldindata.org/technology-adoption

In [26]:
books_df = pd.read_csv('new-books-per-million.csv', index_col='Entity')

books_df = books_df[books_df['Year'] == 1996]
books_df = books_df.rename(columns={'Book titles per capita (Fink-Jensen 2015)':'Books'})
books_df = books_df.drop(columns=['Code', 'Year'])
books_df = books_df.rename(index={'United States':'United States of America', 'Russia':'Russian Federation'})
books_df = books_df.reset_index()
books_df = books_df.rename(columns={'Entity':'Country'})

print(books_df.shape)
books_df.head()

(48, 2)


,Country,Books
0,Algeria,23.423261
1,Argentina,276.041813
2,Austria,985.472115
3,Belgium,1148.548561
4,Brazil,113.694110


In [27]:
m = folium.Map(location=[40, 0], zoom_start=2, width=1000, height=600)
folium.Choropleth(
    world_data,
    name="choropleth",
    data = books_df,
    columns=["Country", "Books"],
    key_on="feature.properties.name",
    fill_color="Greens",
    fill_opacity=0.75,
    line_opacity=0.2,
    legend_name="Número de libros publicados per cápita durante 1996",
    nan_fill_opacity=0.25,
    bins=5,
    highlight=True
).add_to(m)
folium.LayerControl().add_to(m)
m

https://ourworldindata.org/books